# Parkinson's Disease Prediction Model 2 - Eliminating Class Imbalance

## Importing the dependencies

In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Loading the data

In [66]:
parkinsons_data = pd.read_csv('parkinsons.csv')

## Exploring and Preprocessing the Data

In [67]:
parkinsons_data.shape

(195, 24)

In [68]:
parkinsons_data.sample(5)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
135,phon_R01_S33_2,110.453,127.611,105.554,0.00494,0.000040,0.00244,0.00315,0.00731,0.04128,...,0.06688,0.02529,17.707,1,0.653427,0.706687,-5.333619,0.322044,2.631793,0.228319
65,phon_R01_S17_6,228.969,239.541,113.201,0.00238,0.000010,0.00136,0.00140,0.00408,0.01745,...,0.02827,0.00420,24.547,0,0.305429,0.681811,-7.314237,0.006274,2.118596,0.091546
64,phon_R01_S17_5,229.401,252.221,221.156,0.00205,0.000009,0.00114,0.00113,0.00342,0.01457,...,0.02308,0.00300,26.415,0,0.276850,0.673636,-7.496264,0.056844,2.003032,0.073581
69,phon_R01_S18_4,148.090,162.824,67.343,0.00762,0.000050,0.00467,0.00354,0.01400,0.05428,...,0.10070,0.02431,21.718,1,0.487407,0.727313,-6.261141,0.120956,2.137075,0.141958
101,phon_R01_S24_5,128.451,150.449,75.632,0.01551,0.000120,0.00905,0.00909,0.02716,0.06170,...,0.09669,0.11843,15.060,1,0.639808,0.643327,-4.202730,0.310163,2.638279,0.356881


In [69]:
# Label: 'status'

In [70]:
parkinsons_data['status'].value_counts()

1    147
0     48
Name: status, dtype: int64

In [71]:
parkinsons_data_positive = parkinsons_data[parkinsons_data['status'] == 1]
parkinsons_data_negative = parkinsons_data[parkinsons_data['status'] == 0]

In [72]:
print(parkinsons_data_positive.shape, parkinsons_data_negative.shape)

(147, 24) (48, 24)


In [73]:
parkinsons_data_positive_sample = parkinsons_data_positive.sample(parkinsons_data_negative.shape[0])

In [74]:
parkinsons_df = pd.concat([parkinsons_data_negative, parkinsons_data_positive_sample])

In [75]:
parkinsons_df.shape

(96, 24)

In [76]:
parkinsons_df.sample(5)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
176,phon_R01_S43_6,116.388,129.038,108.970,0.00346,0.000030,0.00169,0.00213,0.00507,0.01725,...,0.02623,0.00415,26.143,0,0.361232,0.763242,-6.016891,0.109256,2.004719,0.174429
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.000030,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
34,phon_R01_S07_5,203.184,211.526,196.160,0.00178,0.000009,0.00094,0.00106,0.00283,0.00958,...,0.01403,0.00065,33.047,0,0.340068,0.741899,-7.964984,0.163519,1.423287,0.044539
21,phon_R01_S05_4,153.880,172.860,78.128,0.00480,0.000030,0.00232,0.00267,0.00696,0.04137,...,0.06062,0.01849,19.493,1,0.629017,0.698951,-5.186960,0.237622,2.557536,0.259765
194,phon_R01_S50_6,214.289,260.277,77.973,0.00567,0.000030,0.00295,0.00317,0.00885,0.01884,...,0.03078,0.04398,21.209,0,0.462803,0.664357,-5.724056,0.190667,2.555477,0.148569


## Splitting the features and the label

In [77]:
X = parkinsons_df.drop(columns=['name', 'status'])
y = parkinsons_df['status']

## Splitting the data into training and test sets

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

## Feature scaling the training and test sets

In [79]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

## Training the models and comparing accuracy scores

In [80]:
classifiers = [
    {
        'name': 'Logistic Regression',
        'classifier': LogisticRegression(),
        'params': {
            'solver': ['liblinear', 'lbfgs'],
            'C': [0.1, 1.0, 10.0]
        },
        'train_data': 'X_train_scaled'
    },
    {
        'name': 'Support Vector Classifier',
        'classifier': SVC(),
        'params': {
            'kernel': ['linear', 'sigmoid', 'rbf'],
            'C': [0.1, 1.0, 10.0]
        },
        'train_data': 'X_train_scaled'
    },
    {
        'name': 'Decision Tree Classifier',
        'classifier': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10]
        },
        'train_data': 'X_train'
    },
    {
        'name': 'Random Forest Classifier',
        'classifier': RandomForestClassifier(),
        'params': {
            'max_depth': [None, 5, 10],
            'n_estimators': [50, 100, 200]
        },
        'train_data': 'X_train'
    },
    {
        'name': 'Gaussian Naive Bayes',
        'classifier': GaussianNB(),
        'params': {},
        'train_data': 'X_train'
    },
    {
        'name': 'XGB Classifier',
        'classifier': XGBClassifier(),
        'params': {},
        'train_data': 'X_train'
    },
    {
        'name': 'CatBoost Classifier',
        'classifier': CatBoostClassifier(),
        'params': {},
        'train_data': 'X_train'
    }
]

In [81]:
results_df = pd.DataFrame(columns=['Classifier', 'Best Parameters', 'Accuracy'])

In [82]:
for classifier in classifiers:
    if classifier['train_data'] == 'X_train':
        X_train_data = X_train
    elif classifier['train_data'] == 'X_train_scaled':
        X_train_data = X_train_scaled
    else:
        raise ValueError("Invalid 'train_data'")
        
    grid_search = GridSearchCV(classifier['classifier'], classifier['params'], cv=5)
    grid_search.fit(X_train_data, y_train)
    best_classifier = grid_search.best_estimator_
    if classifier['train_data'] == 'X_train':
        y_pred = best_classifier.predict(X_test)
    elif classifier['train_data'] == 'X_train_scaled':
        X_test_scaled = scaler.transform(X_test)
        y_pred = best_classifier.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    results_df = results_df.append({
        'Classifier': classifier['name'],
        'Best Parameters': grid_search.best_params_,
        'Accuracy': accuracy
    }, ignore_index=True)
    

C:\Users\anupa\AppData\Local\Temp\ipykernel_23596\1338530075.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\anupa\AppData\Local\Temp\ipykernel_23596\1338530075.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\anupa\AppData\Local\Temp\ipykernel_23596\1338530075.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\anupa\AppData\Local\Temp\ipykernel_23596\1338530075.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\anupa\AppData\Local\Temp\ipykernel_23596\133853

Learning rate set to 0.003099
0:	learn: 0.6900916	total: 148ms	remaining: 2m 28s
1:	learn: 0.6872434	total: 150ms	remaining: 1m 14s
2:	learn: 0.6844229	total: 152ms	remaining: 50.5s
3:	learn: 0.6816769	total: 154ms	remaining: 38.3s
4:	learn: 0.6795722	total: 156ms	remaining: 31s
5:	learn: 0.6768057	total: 157ms	remaining: 26.1s
6:	learn: 0.6746708	total: 160ms	remaining: 22.7s
7:	learn: 0.6718879	total: 162ms	remaining: 20.1s
8:	learn: 0.6694421	total: 164ms	remaining: 18.1s
9:	learn: 0.6675049	total: 166ms	remaining: 16.5s
10:	learn: 0.6648754	total: 169ms	remaining: 15.2s
11:	learn: 0.6618460	total: 170ms	remaining: 14s
12:	learn: 0.6589437	total: 172ms	remaining: 13.1s
13:	learn: 0.6569123	total: 175ms	remaining: 12.3s
14:	learn: 0.6543999	total: 176ms	remaining: 11.6s
15:	learn: 0.6517183	total: 178ms	remaining: 11s
16:	learn: 0.6491655	total: 180ms	remaining: 10.4s
17:	learn: 0.6466956	total: 182ms	remaining: 9.92s
18:	learn: 0.6438323	total: 184ms	remaining: 9.48s
19:	learn: 0.64

238:	learn: 0.3073075	total: 584ms	remaining: 1.86s
239:	learn: 0.3065530	total: 587ms	remaining: 1.86s
240:	learn: 0.3055611	total: 588ms	remaining: 1.85s
241:	learn: 0.3046000	total: 590ms	remaining: 1.85s
242:	learn: 0.3037622	total: 592ms	remaining: 1.84s
243:	learn: 0.3028854	total: 594ms	remaining: 1.84s
244:	learn: 0.3021791	total: 596ms	remaining: 1.84s
245:	learn: 0.3014314	total: 605ms	remaining: 1.85s
246:	learn: 0.3005252	total: 606ms	remaining: 1.85s
247:	learn: 0.2995278	total: 608ms	remaining: 1.84s
248:	learn: 0.2987352	total: 610ms	remaining: 1.84s
249:	learn: 0.2974627	total: 612ms	remaining: 1.84s
250:	learn: 0.2966518	total: 614ms	remaining: 1.83s
251:	learn: 0.2958410	total: 616ms	remaining: 1.83s
252:	learn: 0.2950815	total: 618ms	remaining: 1.82s
253:	learn: 0.2945214	total: 620ms	remaining: 1.82s
254:	learn: 0.2936403	total: 621ms	remaining: 1.81s
255:	learn: 0.2925625	total: 623ms	remaining: 1.81s
256:	learn: 0.2920826	total: 625ms	remaining: 1.81s
257:	learn: 

443:	learn: 0.1822391	total: 966ms	remaining: 1.21s
444:	learn: 0.1818817	total: 968ms	remaining: 1.21s
445:	learn: 0.1816888	total: 970ms	remaining: 1.21s
446:	learn: 0.1812807	total: 972ms	remaining: 1.2s
447:	learn: 0.1809729	total: 974ms	remaining: 1.2s
448:	learn: 0.1806103	total: 976ms	remaining: 1.2s
449:	learn: 0.1801037	total: 977ms	remaining: 1.19s
450:	learn: 0.1799142	total: 979ms	remaining: 1.19s
451:	learn: 0.1794147	total: 981ms	remaining: 1.19s
452:	learn: 0.1790256	total: 983ms	remaining: 1.19s
453:	learn: 0.1786830	total: 985ms	remaining: 1.18s
454:	learn: 0.1783175	total: 987ms	remaining: 1.18s
455:	learn: 0.1778708	total: 989ms	remaining: 1.18s
456:	learn: 0.1776139	total: 991ms	remaining: 1.18s
457:	learn: 0.1772254	total: 992ms	remaining: 1.17s
458:	learn: 0.1768306	total: 994ms	remaining: 1.17s
459:	learn: 0.1763039	total: 996ms	remaining: 1.17s
460:	learn: 0.1759079	total: 997ms	remaining: 1.17s
461:	learn: 0.1754042	total: 999ms	remaining: 1.16s
462:	learn: 0.1

654:	learn: 0.1186235	total: 1.35s	remaining: 709ms
655:	learn: 0.1184181	total: 1.35s	remaining: 707ms
656:	learn: 0.1182262	total: 1.35s	remaining: 705ms
657:	learn: 0.1180474	total: 1.35s	remaining: 703ms
658:	learn: 0.1178131	total: 1.35s	remaining: 701ms
659:	learn: 0.1176627	total: 1.36s	remaining: 699ms
660:	learn: 0.1174136	total: 1.36s	remaining: 697ms
661:	learn: 0.1171537	total: 1.36s	remaining: 695ms
662:	learn: 0.1169011	total: 1.36s	remaining: 693ms
663:	learn: 0.1166422	total: 1.36s	remaining: 691ms
664:	learn: 0.1164049	total: 1.37s	remaining: 689ms
665:	learn: 0.1162186	total: 1.37s	remaining: 686ms
666:	learn: 0.1160239	total: 1.37s	remaining: 684ms
667:	learn: 0.1157638	total: 1.37s	remaining: 682ms
668:	learn: 0.1155126	total: 1.37s	remaining: 680ms
669:	learn: 0.1152493	total: 1.38s	remaining: 677ms
670:	learn: 0.1149803	total: 1.38s	remaining: 675ms
671:	learn: 0.1146872	total: 1.38s	remaining: 673ms
672:	learn: 0.1143855	total: 1.38s	remaining: 671ms
673:	learn: 

...

396:	learn: 0.1803126	total: 804ms	remaining: 1.22s
397:	learn: 0.1798643	total: 807ms	remaining: 1.22s
398:	learn: 0.1793599	total: 809ms	remaining: 1.22s
399:	learn: 0.1787572	total: 812ms	remaining: 1.22s
400:	learn: 0.1783872	total: 814ms	remaining: 1.22s
401:	learn: 0.1778506	total: 816ms	remaining: 1.21s
402:	learn: 0.1772821	total: 819ms	remaining: 1.21s
403:	learn: 0.1769474	total: 821ms	remaining: 1.21s
404:	learn: 0.1765478	total: 834ms	remaining: 1.22s
405:	learn: 0.1760988	total: 836ms	remaining: 1.22s
406:	learn: 0.1755585	total: 838ms	remaining: 1.22s
407:	learn: 0.1751255	total: 840ms	remaining: 1.22s
408:	learn: 0.1747076	total: 842ms	remaining: 1.22s
409:	learn: 0.1741964	total: 844ms	remaining: 1.22s
410:	learn: 0.1737408	total: 846ms	remaining: 1.21s
411:	learn: 0.1732792	total: 849ms	remaining: 1.21s
412:	learn: 0.1729057	total: 851ms	remaining: 1.21s
413:	learn: 0.1725597	total: 853ms	remaining: 1.21s
414:	learn: 0.1721530	total: 855ms	remaining: 1.21s
415:	learn: 

572:	learn: 0.1215114	total: 1.19s	remaining: 885ms
573:	learn: 0.1212580	total: 1.19s	remaining: 884ms
574:	learn: 0.1210464	total: 1.19s	remaining: 882ms
575:	learn: 0.1208733	total: 1.19s	remaining: 880ms
576:	learn: 0.1206409	total: 1.2s	remaining: 878ms
577:	learn: 0.1203913	total: 1.2s	remaining: 876ms
578:	learn: 0.1201883	total: 1.2s	remaining: 874ms
579:	learn: 0.1199819	total: 1.2s	remaining: 872ms
580:	learn: 0.1198098	total: 1.21s	remaining: 870ms
581:	learn: 0.1194845	total: 1.21s	remaining: 868ms
582:	learn: 0.1191627	total: 1.21s	remaining: 866ms
583:	learn: 0.1189399	total: 1.21s	remaining: 863ms
584:	learn: 0.1187116	total: 1.21s	remaining: 861ms
585:	learn: 0.1185025	total: 1.22s	remaining: 859ms
586:	learn: 0.1183096	total: 1.22s	remaining: 857ms
587:	learn: 0.1180951	total: 1.22s	remaining: 855ms
588:	learn: 0.1177627	total: 1.22s	remaining: 852ms
589:	learn: 0.1174997	total: 1.22s	remaining: 850ms
590:	learn: 0.1172200	total: 1.23s	remaining: 848ms
591:	learn: 0.11

768:	learn: 0.0849668	total: 1.57s	remaining: 473ms
769:	learn: 0.0848500	total: 1.58s	remaining: 472ms
770:	learn: 0.0846714	total: 1.58s	remaining: 470ms
771:	learn: 0.0845678	total: 1.58s	remaining: 468ms
772:	learn: 0.0844653	total: 1.59s	remaining: 466ms
773:	learn: 0.0843446	total: 1.59s	remaining: 464ms
774:	learn: 0.0841887	total: 1.59s	remaining: 462ms
775:	learn: 0.0840685	total: 1.59s	remaining: 460ms
776:	learn: 0.0839815	total: 1.6s	remaining: 458ms
777:	learn: 0.0838344	total: 1.6s	remaining: 456ms
778:	learn: 0.0836920	total: 1.6s	remaining: 454ms
779:	learn: 0.0835985	total: 1.6s	remaining: 452ms
780:	learn: 0.0834288	total: 1.6s	remaining: 450ms
781:	learn: 0.0832544	total: 1.6s	remaining: 448ms
782:	learn: 0.0831204	total: 1.61s	remaining: 446ms
783:	learn: 0.0830092	total: 1.61s	remaining: 443ms
784:	learn: 0.0828503	total: 1.61s	remaining: 441ms
785:	learn: 0.0827226	total: 1.61s	remaining: 439ms
786:	learn: 0.0825654	total: 1.61s	remaining: 437ms
787:	learn: 0.0824

961:	learn: 0.0629575	total: 1.96s	remaining: 77.3ms
962:	learn: 0.0628747	total: 1.96s	remaining: 75.3ms
963:	learn: 0.0627997	total: 1.96s	remaining: 73.2ms
964:	learn: 0.0627175	total: 1.96s	remaining: 71.2ms
965:	learn: 0.0626623	total: 1.97s	remaining: 69.2ms
966:	learn: 0.0625926	total: 1.97s	remaining: 67.2ms
967:	learn: 0.0624999	total: 1.97s	remaining: 65.1ms
968:	learn: 0.0624210	total: 1.97s	remaining: 63.1ms
969:	learn: 0.0623677	total: 1.99s	remaining: 61.5ms
970:	learn: 0.0622875	total: 1.99s	remaining: 59.4ms
971:	learn: 0.0621980	total: 1.99s	remaining: 57.5ms
972:	learn: 0.0621159	total: 2s	remaining: 55.4ms
973:	learn: 0.0620378	total: 2s	remaining: 53.4ms
974:	learn: 0.0619308	total: 2s	remaining: 51.3ms
975:	learn: 0.0618013	total: 2s	remaining: 49.3ms
976:	learn: 0.0617121	total: 2s	remaining: 47.2ms
977:	learn: 0.0616053	total: 2.01s	remaining: 45.2ms
978:	learn: 0.0615187	total: 2.01s	remaining: 43.1ms
979:	learn: 0.0614519	total: 2.01s	remaining: 41.1ms
980:	lea

C:\Users\anupa\AppData\Local\Temp\ipykernel_23596\1338530075.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [83]:
results_df.sort_values(by='Accuracy', ascending=False, inplace=True)

In [84]:
results_df

,Classifier,Best Parameters,Accuracy
6,CatBoost Classifier,{},0.95
3,Random Forest Classifier,"{'max_depth': None, 'n_estimators': 200}",0.90
4,Gaussian Naive Bayes,{},0.90
5,XGB Classifier,{},0.90
0,Logistic Regression,"{'C': 10.0, 'solver': 'liblinear'}",0.80
1,Support Vector Classifier,"{'C': 1.0, 'kernel': 'linear'}",0.80
2,Decision Tree Classifier,"{'max_depth': None, 'min_samples_split': 2}",0.75
